In [180]:
url = "https://www.keejob.com"

In [19]:
import requests
from bs4 import BeautifulSoup
def get_posts_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    posts = soup.select('div.block_white_a.post.clearfix.silver-job-block > div.content.row-fluid > div.span8')
    links = [post.find('a')['href'] for post in posts if post.find('a')]
    return links

In [21]:
get_posts_links(url)

['/offres-emploi/211433/acheteuse/',
 '/offres-emploi/210508/technicien-support/',
 '/offres-emploi/210308/charg%C3%A9e-de-service-client%C3%A8le-sousse/',
 '/offres-emploi/211206/conseillers-client/',
 '/offres-emploi/210372/covertelesales-manager/',
 '/offres-emploi/211166/assistant-responsable-magasin-strass-sfax/',
 '/offres-emploi/210518/charg%C3%A9-paie/',
 '/offres-emploi/210615/juriste-en-droit-priv%C3%A9/',
 '/offres-emploi/210626/magasinier-pi%C3%A8ces-de-rechange-automobile/',
 '/offres-emploi/211434/ing%C3%A9nieur-maintenance-pec-med/',
 '/offres-emploi/211205/charg%C3%A9e-de-sourcing-sousse/',
 '/offres-emploi/209839/charg%C3%A9e-de-sourcing-en-recrutement-sousse/',
 '/offres-emploi/210251/operatori-call-center-di-lingua-italiana/']

In [60]:
def get_post_content(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.text, "html.parser")
    content = soup.find('div', {'id': 'main', 'class': 'span12 page image-preloader'})
    return content
    
    

In [ ]:
get_post_content(url+'211433/acheteuse/')

In [106]:
import re


def get_company_details(content):
    block = content.find("div", {"class": "span9 content"}).text
    text = re.sub(r"\t+", "", block)
    text = text.strip().split("\n")
    text = [line for line in text if line != ""]
    company = text[0]
    sector = text[2]
    size = text[4]
    
    return {
        "company": company,
        "sector": sector,
        "size": size
    }
    # print(company)

In [107]:
get_company_details(get_post_content(url+'211433/acheteuse/'))

{'company': 'PLASTIC ELECTROMECHANIC COMPANY PEC',
 'sector': 'industrie électro-mécaniques',
 'size': 'Entre 200 et 500 employés'}

In [108]:
get_company_details(get_post_content(url+'210508/technicien-support/'))

{'company': 'BINITNS',
 'sector': 'consulting / étude / stratégie',
 'size': 'Moins de 20 employés'}

In [161]:
def get_job_details(content):
    job_title = content.find("h2", class_="job-title").text
    description = content.find("div", class_="block_a span12 no-margin-left").text
    details = content.find_all("div", class_="meta")
    data = {}    
    for meta in details:
        # Get the label (text inside the <b> tag)
        label = meta.find('b').text.strip().replace(":", "")
        # Get the value (text after the <br> tag)
        value = meta.find('br').next_sibling.strip() if meta.find('br') else ''
        value = re.sub(r'[\s\n\t]+', ' ', value).strip()
        data[label] = value

    return {
        "title": job_title,
        "description": description,
        "reference": data["Référence"],
        "post_date": data["Publiée le"],
        "job_type": data["Type de poste"],
        "workplace": data["Lieu de travail"],
        "experience": data["Expérience"],
        "degree": data["Étude"],
        "disponibility": data["Disponibilité"],
        "languages": data["Langues"],
    }


In [162]:
get_job_details(get_post_content(url+'211433/acheteuse/'))

{'title': 'Acheteuse',
 'description': "\nDescription de l'annonce:\nDescription des taches :Gérer le processus d’achat de biens et services en respectant le budget et les délais.Évaluer et sélectionner les fournisseurs en fonction des critères de qualité, coût et délais.Négocier les conditions d’achat pour obtenir les meilleures offres.Suivre les commandes et s’assurer de la bonne réception des produitsÉlaborer des cahiers des charges et des appels d’offresProfil Recherché\xa0 :Maîtrise des techniques de négociation et des contrats d’achat.Connaissance des outils de gestion des achats (ERP, logiciels de gestionExcellentes compétences en communication et en travail d’équipe.Capacité à établir des relations de confiance avec les fournisseursFormation : Diplôme en commerce, gestion des achats, logistiqueExpérience : Minimum 3 ans\xa0 d’expérience dans un poste similaire\nAdresse : djebel Ouest -Zaghouan (PEC MED )\xa0 \xa0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\t\t\tImprimer\n\t\t\t

In [193]:
def get_number_of_pages(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    pages = soup.find_all('li', class_='page-item')
    page_number = pages[3].find('a')['aria-label'].split()[-1]
    return int(page_number)

get_number_of_pages('https://www.keejob.com/offres-emploi/')

88

In [183]:
def get_pagited_posts_links_and_content(url, n):
    data = {}
    links = []
    for i in range(1, n+1):
        page_url = url + f"/offres-emploi/?page={i}"
        links += get_posts_links(page_url)
    for i, link in enumerate(links):
        data[link] = get_post_content(url+link)
        if i == 5:
            break
        # print(url+link)
    return data